In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.metrics import balanced_accuracy_score
from imblearn.metrics import classification_report_imbalanced
from sklearn.metrics import confusion_matrix, classification_report
from collections import Counter
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE
from imblearn.combine import SMOTEENN

In [2]:
data_v5 = pd.read_csv("data_v5.csv", low_memory=False)
data_v5

,issue_d,annual_inc,dti,fico_range_low,loan_amnt,open_acc,pub_rec,pub_rec_bankruptcies,tax_liens,loan_status_Fully Paid,home_ownership_ANY,home_ownership_MORTGAGE,home_ownership_NONE,home_ownership_OTHER,home_ownership_OWN,home_ownership_RENT
0,2015-12-01,55000.0,5.91,675.0,3600.0,7.0,0.0,0.0,0.0,1,0,1,0,0,0,0
1,2015-12-01,65000.0,16.06,715.0,24700.0,22.0,0.0,0.0,0.0,1,0,1,0,0,0,0
2,2015-12-01,63000.0,10.78,695.0,20000.0,6.0,0.0,0.0,0.0,1,0,1,0,0,0,0
3,2015-12-01,104433.0,25.37,695.0,10400.0,12.0,0.0,0.0,0.0,1,0,1,0,0,0,0
4,2015-12-01,34000.0,10.20,690.0,11950.0,5.0,0.0,0.0,0.0,1,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1344234,2016-10-01,130000.0,20.59,735.0,18000.0,17.0,0.0,0.0,0.0,1,0,0,0,0,1,0
1344235,2016-10-01,180792.0,22.03,705.0,29400.0,16.0,0.0,0.0,0.0,1,0,1,0,0,0,0
1344236,2016-10-01,157000.0,10.34,735.0,32000.0,14.0,0.0,0.0,0.0,0,0,1,0,0,0,0
1344237,2016-10-01,150000.0,12.25,665.0,16000.0,12.0,4.0,3.0,0.0,1,0,0,0,0,0,1


In [3]:
data_v5.describe()

,annual_inc,dti,fico_range_low,loan_amnt,open_acc,pub_rec,pub_rec_bankruptcies,tax_liens,loan_status_Fully Paid,home_ownership_ANY,home_ownership_MORTGAGE,home_ownership_NONE,home_ownership_OTHER,home_ownership_OWN,home_ownership_RENT
count,1.344239e+06,1.344239e+06,1.344239e+06,1.344239e+06,1.344239e+06,1.344239e+06,1.344239e+06,1.344239e+06,1.344239e+06,1.344239e+06,1.344239e+06,1.344239e+06,1.344239e+06,1.344239e+06,1.344239e+06
mean,7.627233e+04,1.828653e+01,6.961737e+02,1.442149e+04,1.159528e+01,2.153977e-01,1.344515e-01,5.216632e-02,8.003554e-01,2.127598e-04,4.947677e-01,3.347619e-05,1.071238e-04,1.076713e-01,3.972076e-01
std,6.990932e+04,1.116102e+01,3.184432e+01,8.716773e+03,5.473660e+00,6.020465e-01,3.779437e-01,3.980553e-01,3.997333e-01,1.458474e-02,4.999728e-01,5.785767e-03,1.034951e-02,3.099650e-01,4.893198e-01
min,1.600000e+01,-1.000000e+00,6.600000e+02,5.000000e+02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,4.590200e+04,1.180000e+01,6.700000e+02,8.000000e+03,8.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,6.500000e+04,1.762000e+01,6.900000e+02,1.200000e+04,1.100000e+01,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
75%,9.000000e+04,2.406000e+01,7.100000e+02,2.000000e+04,1.400000e+01,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00
max,1.099920e+07,9.990000e+02,8.450000e+02,4.000000e+04,9.000000e+01,8.600000e+01,1.200000e+01,8.500000e+01,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00


In [4]:
data_v5.dtypes

issue_d                     object
annual_inc                 float64
dti                        float64
fico_range_low             float64
loan_amnt                  float64
open_acc                   float64
pub_rec                    float64
pub_rec_bankruptcies       float64
tax_liens                  float64
loan_status_Fully Paid       int64
home_ownership_ANY           int64
home_ownership_MORTGAGE      int64
home_ownership_NONE          int64
home_ownership_OTHER         int64
home_ownership_OWN           int64
home_ownership_RENT          int64
dtype: object

In [5]:
#Setting y/prediction variable column
y = data_v5['loan_status_Fully Paid']
#y.ravel()
y

0          1
1          1
2          1
3          1
4          1
          ..
1344234    1
1344235    1
1344236    0
1344237    1
1344238    0
Name: loan_status_Fully Paid, Length: 1344239, dtype: int64

In [6]:
#Setting x/dependent variables 
X = data_v5[[
'annual_inc','fico_range_low', 'dti', 'loan_amnt',
'open_acc',
'pub_rec',
'pub_rec_bankruptcies',
'tax_liens', 'home_ownership_ANY','home_ownership_MORTGAGE','home_ownership_NONE','home_ownership_OTHER','home_ownership_OWN','home_ownership_RENT']]
X

,annual_inc,fico_range_low,dti,loan_amnt,open_acc,pub_rec,pub_rec_bankruptcies,tax_liens,home_ownership_ANY,home_ownership_MORTGAGE,home_ownership_NONE,home_ownership_OTHER,home_ownership_OWN,home_ownership_RENT
0,55000.0,675.0,5.91,3600.0,7.0,0.0,0.0,0.0,0,1,0,0,0,0
1,65000.0,715.0,16.06,24700.0,22.0,0.0,0.0,0.0,0,1,0,0,0,0
2,63000.0,695.0,10.78,20000.0,6.0,0.0,0.0,0.0,0,1,0,0,0,0
3,104433.0,695.0,25.37,10400.0,12.0,0.0,0.0,0.0,0,1,0,0,0,0
4,34000.0,690.0,10.20,11950.0,5.0,0.0,0.0,0.0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1344234,130000.0,735.0,20.59,18000.0,17.0,0.0,0.0,0.0,0,0,0,0,1,0
1344235,180792.0,705.0,22.03,29400.0,16.0,0.0,0.0,0.0,0,1,0,0,0,0
1344236,157000.0,735.0,10.34,32000.0,14.0,0.0,0.0,0.0,0,1,0,0,0,0
1344237,150000.0,665.0,12.25,16000.0,12.0,4.0,3.0,0.0,0,0,0,0,0,1


In [38]:
# split Data into training and testing sets **NOT USING STRATIFY method**
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [39]:
Counter(y_train)

Counter({0: 201176, 1: 807003})

In [40]:
# SMOTEENN module
smote_enn = SMOTEENN(random_state=1)
X_resampled, y_resampled = smote_enn.fit_resample(X, y)

In [41]:
Counter(y_resampled)

Counter({0: 614988, 1: 465924})

In [42]:
# Creating a StandardScaler instance.
scaler = StandardScaler()
# Fitting the Standard Scaler with the training data.
X_scaler = scaler.fit(X_train)

# Scaling the data.
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)


In [43]:
# Create a random forest classifier.
rf_model = RandomForestClassifier(n_estimators=128, random_state=1) 


In [54]:
len(X_train_scaled)


1008179

In [55]:
len(y_resampled)

1080912

In [47]:
# Fitting the model
rf_model = rf_model.fit(X_train_scaled, y_resampled)


ValueError: Found input variables with inconsistent numbers of samples: [1008179, 1080912]

In [45]:
# Making predictions using the testing data.
predictions = rf_model.predict(X_test_scaled)


In [46]:
# Calculating the confusion matrix
cm = confusion_matrix(y_test, predictions)

# Create a DataFrame from the confusion matrix.
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])

cm_df

,Predicted 0,Predicted 1
Actual 0,3424,63770
Actual 1,6974,261892


In [30]:
# Calculating the accuracy score.
acc_score = accuracy_score(y_test, predictions)
acc_score

0.7894899720288043

In [31]:
# Displaying results
print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report(y_test, predictions))

Confusion Matrix


,Predicted 0,Predicted 1
Actual 0,3424,63770
Actual 1,6974,261892


Accuracy Score : 0.7894899720288043
Classification Report
              precision    recall  f1-score   support

           0       0.33      0.05      0.09     67194
           1       0.80      0.97      0.88    268866

    accuracy                           0.79    336060
   macro avg       0.57      0.51      0.48    336060
weighted avg       0.71      0.79      0.72    336060



In [14]:
# Displaying results
#print("Confusion Matrix")
#display(cm_df)
#print(f"Accuracy Score : {acc_score}")
#print("Classification Report")
#print(classification_report(y_test, predictions))

Confusion Matrix


,Predicted 0,Predicted 1
Actual 0,3401,63692
Actual 1,7056,261911


Accuracy Score : 0.7894780693923704
Classification Report
              precision    recall  f1-score   support

           0       0.33      0.05      0.09     67093
           1       0.80      0.97      0.88    268967

    accuracy                           0.79    336060
   macro avg       0.56      0.51      0.48    336060
weighted avg       0.71      0.79      0.72    336060



In [15]:
# Calculate feature importance in the Random Forest model.
importances = rf_model.feature_importances_
importances

array([0.22963389, 0.07954319, 0.33753019, 0.19550105, 0.1383308 ,
       0.00908171, 0.00562779, 0.00475139])

In [16]:
# We can sort the features by their importance.
sorted(zip(rf_model.feature_importances_, X.columns), reverse=True)

[(0.3375301925184225, 'dti'),
 (0.22963388846618074, 'annual_inc'),
 (0.19550104632852985, 'loan_amnt'),
 (0.1383308004691008, 'open_acc'),
 (0.07954318686115203, 'fico_range_low'),
 (0.009081711751419752, 'pub_rec'),
 (0.005627786129967219, 'pub_rec_bankruptcies'),
 (0.004751387475227239, 'tax_liens')]